In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install cellpose==3.0.8

In [ ]:
FOLDER_NAME = "annotated"

!unzip "/content/drive/MyDrive/{FOLDER_NAME}.zip" -d "/content"

In [ ]:
import numpy as np
import glob
import cv2
from torch.utils.data import Dataset, random_split

file_names = glob.glob(FOLDER_NAME + "/*.jpg")

class ImageDataset(Dataset):
  def __len__(self):
    return len(file_names)

  def __getitem__(self, idx):
    img = cv2.imread(file_names[idx], 0)
    mask = np.load(file_names[idx].replace(".jpg", "_seg.npy"), allow_pickle=True).item()["masks"]
    return img, mask

dataset = ImageDataset()

training_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

In [ ]:
# Getting training and test data into memory.
# In future maybe an own training loop can be implemented if memory is not enough.
training_images, training_masks = [], []

for image, mask in training_dataset:
  training_images.append(image)
  training_masks.append(mask)

test_images, test_masks = [], []
for image, mask in test_dataset:
  test_images.append(image)
  test_masks.append(mask)

In [ ]:
from cellpose import models, train, utils, transforms, dynamics
from tqdm import trange
import torch
import logging

def train_model(model_type, n_epochs):
  logger = logging.getLogger('cellpose')
  logger.setLevel(logging.DEBUG)

  model = models.CellposeModel(model_type=model_type, gpu=torch.cuda.is_available())
  test_labels = dynamics.labels_to_flows(test_masks, device=model.net.device)

  train.train_seg(
              model.net, n_epochs=n_epochs,
              train_data=training_images, train_labels=training_masks,
              test_data=test_images, test_labels=test_masks,
              channels=[0, 0], normalize=True,
              weight_decay=1e-4, SGD=True, learning_rate=0.1,
              save_path="/content/drive/MyDrive/cellpose-cell-counting", model_name=f"{model_type}")

In [ ]:
train_model("cyto3", n_epochs = 1000)

INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> loading model /root/.cellpose/models/cyto3
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 3/3 [00:01<00:00,  1.77it/s]
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 13/13 [00:08<00:00,  1.54it/s]
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 3/3 [00:01<00:00,  1.72it/s]
INFO:cellpose.train:>>> computing diameters
100%|██████████| 3/3 [00:00<00:00, 220.19it/s]
INFO:cellpose.train:>>> using channels [0, 0]
INFO:cellpose.train:>>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': False, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
INFO:cellpose.train:>>> n_epochs=